In [111]:
import numpy as np
import copy
from collections import defaultdict

In [112]:
r1 = np.array([[1,1,1,1]], dtype=int)
r2 = np.array([[0,1,0],[1,1,1],[0,1,0]], dtype=int)
r3 = np.array([[1,1,1],[0,0,1],[0,0,1]], dtype=int)
r4 = np.array([[1],[1],[1],[1]], dtype=int)
r5 = np.array([[1,1],[1,1]], dtype=int)

In [113]:
def principal_period(s):
    i = (s+s).find(s, 1, -1)
    return None if i == -1 else s[:i]

In [297]:
def print_cave(cave):
    for r in range(cave.shape[0]-1, -1, -1):
        s = ['.' if c == 0 else '#' for c in cave[r,:]]
        print(''.join(s))

class Rock():
    def __init__(self, form, x, y):
        self.form = form.copy()
        self.x = x # X of lower left corner
        self.y = y # y of lower left corner
        
        self.h, self.w = form.shape
        
    @property
    def l(self):
        return self.x

    @property
    def b(self):
        return self.y

    @property
    def r(self):
        return self.x + self.w

    @property
    def t(self):
        return self.y + self.h


class Cave():
    def __init__(self):
        self.rocks = [r1, r2, r3, r4, r5]
        self.top = 1
        self.total_top = 0
        self.bottom = [0, 0, 0, 0, 0, 0, 0]

        self.resetCave()
        self.rock = None
        
        self.rock_counter = 0
        
    def resetCave(self):
        self.cave = np.zeros((2000*4 + 1,9), dtype=int)
        self.cave[0,:] = 1
        self.cave[:,0] = 1
        self.cave[:,-1] = 1

    def addRock(self):
        # print('Add top', self.top)
        if self.rock_counter > 4:
            self.rock_counter = 0
            
        self.rock = Rock(self.rocks[self.rock_counter], 3, self.top+3)

        self.rock_counter += 1
        
        # tcave = self.cave.copy()
        # tcave[self.rock.b:self.rock.t,self.rock.l:self.rock.r] += self.rock.form
        # print_cave(tcave)

    def push(self, d):
        # print('Push')
        if d == '>':
            if np.all(self.cave[self.rock.b:self.rock.t,self.rock.l+1:self.rock.r+1]+self.rock.form < 2):
                self.rock.x += 1
        else:
            if np.all(self.cave[self.rock.b:self.rock.t,self.rock.l-1:self.rock.r-1]+self.rock.form < 2):
                self.rock.x -= 1
        # tcave = self.cave.copy()
        # tcave[self.rock.b:self.rock.t,self.rock.l:self.rock.r] += self.rock.form
        # print_cave(tcave)
            
    def drop(self):
        # print('Drop')
        reset = False
        if np.any(self.cave[self.rock.b-1:self.rock.t-1,self.rock.l:self.rock.r]+self.rock.form > 1):
            self.cave[self.rock.b:self.rock.t,self.rock.l:self.rock.r] += self.rock.form
            if self.rock.t > self.top:
                self.top = self.rock.t
                
            if self.rock.b > 2:
                tcave = self.cave.copy()
                tcave[self.rock.b:self.rock.t,self.rock.l:self.rock.r] += self.rock.form
            
                cavefloor = tcave[self.rock.b-1:self.rock.b+1,:]
                # print(cavefloor)
                # print(self.rock.form)
                sealed = True
                for i in range(9):
                    if np.all(cavefloor[:,i] < 1):
                        sealed = False
                        break
                if sealed:
                    # print('Sealed!', self.top, self.total_top)
                    # print('Removing', self.rock.b )
                    # print_cave(cavefloor)
                    self.resetCave()
                    self.cave[0:(self.top - self.rock.b+1),:] = tcave[(self.rock.b-1):self.top,:]
                    self.total_top += self.rock.b-1
                    self.top -= self.rock.b-1
                    reset = True
            # print_cave(tcave)
            # print('Height', self.top)
            return True, reset

        self.rock.y -= 1
        
        # tcave = self.cave.copy()
        # tcave[self.rock.b:self.rock.t,self.rock.l:self.rock.r] += self.rock.form
        # print_cave(tcave)
        return False, False


In [313]:
with open('input17.txt', 'r') as inf:
    stream_pattern = inf.readline().strip()
    
# print(stream_pattern)
streams = []
c = Cave()
# print(c.cave)
patterns=defaultdict(list)      

i=1
s=1
max_rocks=1000000000000

while i <= max_rocks:
    c.addRock()
    i += 1
    done = False
    while not done:
        if len(streams) == 0:
            streams = [s for s in stream_pattern]
            s=1
        c.push(streams.pop(0))
        s+=1
        done, sealed = c.drop()
        
        if sealed:
            key = (hash(c.cave[0:c.top,:].tobytes()), s, c.rock_counter)
            patterns[key].append((i, c.top, c.total_top))
            
            if len(patterns[key]) > 1:
                #print('Pattern repeat!', i, key[1], key[2])
                plen = i - patterns[key][-2][0]
                if plen < max_rocks - i:
                    n_repeats = (max_rocks - i) // plen
                    i += n_repeats * plen
                    c.total_top += (c.total_top - patterns[key][-2][2]) * n_repeats
            
        
        
print(c.total_top+c.top-1)



1566272189352
